# 13.Recurrent Neural Network

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.layers as L
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [2]:
# Hyper parameters
num_epochs = 25
num_classes = 10
batch_size = 512
learning_rate = 0.001

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print("training_data: ", x_train.shape)
print("test_data: ", x_test.shape)
print("training_label: ", y_train.shape)
print("test_label: ", y_test.shape)

training_data:  (60000, 28, 28)
test_data:  (10000, 28, 28)
training_label:  (60000,)
test_label:  (10000,)


In [3]:
x_train_eager = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_test_eager = tf.convert_to_tensor(x_test, dtype=tf.float32)
y_train_eager = tf.reshape(tf.one_hot(y_train, 10), (-1, 10))
y_test_eager = tf.reshape(tf.one_hot(y_test, 10), (-1, 10))

print("training_data: ", x_train_eager.shape)
print("test_data: ", x_test_eager.shape)
print("training_label: ", y_train_eager.shape)
print("test_label: ", y_test_eager.shape)

training_data:  (60000, 28, 28)
test_data:  (10000, 28, 28)
training_label:  (60000, 10)
test_label:  (10000, 10)


### DataSet
You make Dataset using `tf.data.Dataset` Class but Keras API doesn't need this dataset. If you write training loop code manually, `Dataset` class is very useful. And using keras API, you need numpy.array inputs instead of tf.Tensor. I don't know why...so you only need numpy preprocessing (or get numpy.array from tf.Tensor using numpy() method after preprocessing using function of tf).

### NOTE
This notebook we don't need 'tf.data.Dataset'. This code only just for reference.

In [4]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices((x_train_eager, y_train_eager))
    .batch(batch_size)
    .shuffle(10000)
)
train_dataset = train_dataset.repeat()

In [5]:
test_dataset = (
    tf.data.Dataset.from_tensor_slices((x_test_eager, y_test_eager))
    .batch(1000)
    .shuffle(10000)
)
test_dataset = test_dataset.repeat()

### RNN using LSTM
In keras API, LSTM recives inputs tensor whose shape is (batch_size, seqence_length, feature_dim), and output tensor whose shape is (batch_size, fearure_dim).When you need all time sequence data, you have to give `return_sequences=True` to LSTM's constractor. Generally, when you stack LSTM's, you need all sequence data.

In [8]:
class RNN(tf.keras.Model):
    def __init__(self, hidden_size=10, num_layers=2, num_classes=10):
        super(RNN, self).__init__(name='mnist_rnn')
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = self.get_lstm_layers(hidden_size, num_layers)            
        self.fc = L.Dense(num_classes, activation="softmax")
    
    @staticmethod
    def get_lstm_layers(hidden_size, num_layers):
        lstm_layers = []
        # we need all sequence data. write return_sequences=True! 
        for i in range(num_layers-1):
            lstm_layers.append(
                L.LSTM(units=hidden_size, return_sequences=True)
            )
        # the final layer return only final sequence
        # if you need all sequences, you have to write return_sequences=True.
        lstm_layers.append(L.LSTM(units=hidden_size))
        return tf.keras.Sequential(lstm_layers)
        
    def call(self, x):        
        # Forward propagate LSTM
        out = self.lstm(x)
        out = self.fc(out)
        return out

In [11]:
model = RNN()
optimizer = tf.keras.optimizers.Adam(learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=["accuracy"])

# Eager Execution initialize parameters when using model.call()
model(x_train_eager[:50])

model.summary()

Model: "mnist_rnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    multiple                  2400      
_________________________________________________________________
dense_2 (Dense)              multiple                  110       
Total params: 2,510
Trainable params: 2,510
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(x=x_train_eager.numpy(), 
          y=y_train_eager.numpy(), 
          validation_split=0.2, 
          epochs=num_epochs,
          batch_size=batch_size)

Train on 48000 samples, validate on 12000 samples
Epoch 1/25
48000/48000 [==============================] - 4s 81us/sample - loss: 2.2043 - accuracy: 0.2040 - val_loss: 2.0070 - val_accuracy: 0.3113
Epoch 2/25
48000/48000 [==============================] - 1s 24us/sample - loss: 1.7273 - accuracy: 0.4334 - val_loss: 1.4141 - val_accuracy: 0.5475
Epoch 3/25
48000/48000 [==============================] - 1s 23us/sample - loss: 1.2682 - accuracy: 0.5857 - val_loss: 1.0611 - val_accuracy: 0.6589
Epoch 4/25
48000/48000 [==============================] - 1s 23us/sample - loss: 0.9947 - accuracy: 0.6739 - val_loss: 0.8703 - val_accuracy: 0.7171
Epoch 5/25
48000/48000 [==============================] - 1s 23us/sample - loss: 0.8464 - accuracy: 0.7215 - val_loss: 0.7709 - val_accuracy: 0.7457
Epoch 6/25
48000/48000 [==============================] - 1s 23us/sample - loss: 0.7627 - accuracy: 0.7479 - val_loss: 0.7020 - val_accuracy: 0.7643
Epoch 7/25
48000/48000 [==============================] 

In [14]:
test_loss, test_acc = model.evaluate(x=x_test_eager.numpy(), 
                                     y=y_test_eager.numpy(), verbose=False)

print("test_accracy: ", test_acc)

test_accracy:  0.8806
